In [1]:
# %pip install --upgrade imutils


In [1]:
import pandas as pd
import cv2
import glob
import matplotlib.pyplot as plt
import imutils
from imutils.contours import sort_contours
import numpy as np
from PIL import Image
import os



In [13]:
TrainSet = pd.read_csv("E:/OCR_project/Datasets/archive/data_Set_converting/TrainLabels.csv")
validSet = pd.read_csv("E:/OCR_project/Datasets/archive/data_Set_converting/ValidateLabels.csv")

TrainSet.head()

,Image,Text
0,0,123456789
1,AHTD3A0001_Para1_3.jpg,طفنا وسعينا مع شيخ. كان جاري في الخيم يتكلم وه...
2,AHTD3A0001_Para1_4.jpg,مثل انقض بغلس له الضابط لزمتك. سألت راجح هل ...
3,AHTD3A0005_Para1_1.jpg,ذهب نوح مظفر ضرغام بصح ب رؤوف بن لؤي رايق ظافر...
4,AHTD3A0005_Para1_2.jpg,بدأت قوافل الحجيج حاج اثر اخر يلبي. عند وصولنا...


new

In [6]:
train_image_dir = r"E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed"
val_image_dir   = r"E:/OCR_project/Datasets/archive/Validate_deskewed"

In [14]:
# Train_Images = [cv2.imread('Line Images/'+Name) for Name in TrainSet['Image']]
# Test_Images = [cv2.imread('Line Images/'+Name) for Name in TestSet['Image']]
# Load train images
Train_Images = [cv2.imread(os.path.join(train_image_dir, name)) for name in TrainSet['Image']]

# Load validation/test images
validation_Images = [cv2.imread(os.path.join(val_image_dir, name)) for name in validSet['Image']]

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 1049040 bytes in function 'cv::OutOfMemoryError'


In [8]:
Train_Labels = TrainSet['Text']
Test_Labels = TestSet['Text']

In [9]:
# def augment_image(img,label):
#     # Making directory for saving the augmented images along with the original image
    
#     dirname = 'AugmentedImages'
#     os.makedirs('AugmentedImages', exist_ok=True)  
#     #Saving the original Image
#     path0 = 'AugmentedImages/' + label
#     cv2.imwrite(path0,img)
    
#     #saving the blurred image
#     blurred = cv2.GaussianBlur(img,(91,91),cv2.BORDER_DEFAULT)
#     path1 = 'AugmentedImages/blurred' +label
#     cv2.imwrite(path1,blurred)
    
#     #saving the edged image
#     edged = cv2.Canny(blurred, 30, 150)
#     path2 = 'AugmentedImages/edged' +label
#     cv2.imwrite(path2,edged)
    
#     #saving the bleed through enhanced img
    
#     dst = cv2.fastNlMeansDenoisingColored(img,None,100,100,7,21)
#     path3 = 'AugmentedImages/bte' +label
#     cv2.imwrite(path3,dst)
    
#     #saving the 25% resolution off
    
#     PIL_image = Image.fromarray(np.uint8(img)).convert('RGB')
    
#     path4 = 'AugmentedImages/lq' +label
#     PIL_image.save(path4, quality=75)
    
#     #saving the contoured version of the text line
    
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  

#     #--- performing Otsu threshold ---
#     ret,thresh1 = cv2.threshold(gray, 0, 255,cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)

#     #--- choosing the right kernel
#     #--- kernel size of 3 rows (to join dots above letters 'i' and 'j')
#     #--- and 10 columns to join neighboring letters in words and neighboring words
#     rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (70, 20))
#     dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)
#     contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

#     im2 = img.copy()
#     for cnt in contours:
#             x, y, w, h = cv2.boundingRect(cnt)
#             if w>100 and h>30:
#                 cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 3)    
    
#     path5 = 'AugmentedImages/contoured' +label
#     cv2.imwrite(path5,im2)

import cv2
import os
import numpy as np
from PIL import Image

def augment_image(img, label):
    dirname = 'AugmentedImages'
    os.makedirs(dirname, exist_ok=True)

    # Clean filename
    base_label = os.path.basename(label)

    # 1. Original image
    path0 = os.path.join(dirname, base_label)
    cv2.imwrite(path0, img)

    # 2. Blurred image
    blurred = cv2.GaussianBlur(img, (91, 91), cv2.BORDER_DEFAULT)
    path1 = os.path.join(dirname, "blurred_" + base_label)
    cv2.imwrite(path1, blurred)

    # 3. Edged image
    edged = cv2.Canny(blurred, 30, 150)
    path2 = os.path.join(dirname, "edged_" + base_label)
    cv2.imwrite(path2, edged)

    # 4. Bleed-through enhanced
    if len(img.shape) == 3 and img.shape[2] == 3:
        dst = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
    else:
        dst = cv2.fastNlMeansDenoising(img, None, 10, 7, 21)
    path3 = os.path.join(dirname, "bte_" + base_label)
    cv2.imwrite(path3, dst)

    # 5. Lower quality version (25% resolution loss)
    PIL_image = Image.fromarray(np.uint8(img)).convert('RGB')
    path4 = os.path.join(dirname, "lq_" + base_label)
    PIL_image.save(path4, quality=75)

    # 6. Contoured version
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (70, 20))
    dilation = cv2.dilate(thresh1, rect_kernel, iterations=1)

    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    im2 = img.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 100 and h > 30:
            cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 3)

    path5 = os.path.join(dirname, "contoured_" + base_label)
    cv2.imwrite(path5, im2)


In [11]:
print(Train_Images[:10])
# print(TrainSet['Image'][:10])


[None, None, None, None, None, None, None, None, None, None]


In [12]:
for i in range(len(Train_Images)):
    augment_image(Train_Images[i],TrainSet['Image'][i])

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:798: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [32]:
import os
import cv2

# Use raw string to avoid Windows backslash issues
train_image_dir = r"E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed"

# Build full paths and load images
Train_Images = []
for name in TrainSet['Image']:
    image_path = os.path.join(train_image_dir, name)
    if not os.path.exists(image_path):
        print("[ERROR] File does not exist:", image_path)
        Train_Images.append(None)
    else:
        img = cv2.imread(image_path)
        if img is None:
            print("[ERROR] OpenCV failed to read:", image_path)
        Train_Images.append(img)


[ERROR] File does not exist: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0001_Para2_3.png
[ERROR] File does not exist: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0001_Para2_4.png
[ERROR] File does not exist: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0001_Para3_1.png
[ERROR] File does not exist: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0001_Para3_2.png
[ERROR] File does not exist: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0002_Para2_1.png
[ERROR] File does not exist: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0002_Para2_2.png
[ERROR] File does not exist: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0002_Para2_3.png
[ERROR] File does not exist: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0002_Para2_4.png
[ERROR] File does not exist: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskew

"start here
"

In [1]:
import torch
print(torch.__version__)


2.6.0+cu118


In [2]:
import os
import pandas as pd
import cv2
from torch.utils.data import Dataset, DataLoader

# === Load CSVs ===
train_df = pd.read_csv("TrainLabels.csv")
val_df = pd.read_csv("ValidateLabels.csv")

# === Your image paths ===
train_image_dir = "E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed/"
val_image_dir = "E:/OCR_project/Datasets/archive/Validate_deskewed/Validate_deskewed/"


In [3]:
class OCRDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['Image'])
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)

        if img is None:
            raise ValueError(f"Could not read image: {img_path}")

        if self.transform:
            img = self.transform(img)

        label = row['Text']
        return img, label


Create DataLoaders

In [7]:
train_dataset = OCRDataset(train_df, train_image_dir)
val_dataset = OCRDataset(val_df, val_image_dir)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [15]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

# === CONFIG ===
train_csv_path = r"E:/OCR_project/Datasets/archive/data_Set_converting/TrainLabels.csv"
train_img_dir = r"E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed"

# === STEP 1: Load CSV ===
df = pd.read_csv(train_csv_path)

# === STEP 2: Filter out invalid rows ===
df = df[df['Image'].astype(str).str.lower() != '0']   # remove '0'
df = df[df['Image'].notna()]                          # remove NaN
df = df[df['Image'].str.endswith('.jpg')]             # only .jpg

df = df.reset_index(drop=True)
print("Filtered CSV shape:", df.shape)

# === STEP 3: Show first 5 valid images ===
plt.figure(figsize=(15, 6))

for i in range(min(5, len(df))):
    row = df.iloc[i]
    img_filename = row['Image'].strip().replace("\\", "/")  # normalize slashes
    label = row['Text']

    img_path = os.path.join(train_img_dir, img_filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"❌ Could not load image: {img_path}")
        continue

    plt.subplot(1, 5, i + 1)
    plt.imshow(img, cmap='gray')
    plt.title(f"Label:\n{label}", fontsize=8)
    plt.axis('off')

plt.tight_layout()
plt.show()


Filtered CSV shape: (1901, 2)
❌ Could not load image: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0001_Para1_3.jpg
❌ Could not load image: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0001_Para1_4.jpg
❌ Could not load image: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0005_Para1_1.jpg
❌ Could not load image: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0005_Para1_2.jpg
❌ Could not load image: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0005_Para1_3.jpg


<Figure size 1500x600 with 0 Axes>

In [16]:
print("Sample path:", img_path)
print("File exists?", os.path.exists(img_path))


Sample path: E:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed\AHTD3A0005_Para1_3.jpg
File exists? False


In [ ]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

# === CONFIG ===
csv_path = r"E:/OCR_project/Datasets/archive/data_Set_converting/TrainLabels.csv"
base_dir = r"E:/OCR_project/Datasets/archive"

df = pd.read_csv(csv_path)

# Clean and filter
df = df[df['Image'].astype(str).str.lower() != '0']
df = df[df['Image'].notna()]
df = df[df['Image'].str.endswith('.jpg')]
df = df.reset_index(drop=True)

# Show first 5
plt.figure(figsize=(15, 6))

for i in range(min(5, len(df))):
    row = df.iloc[i]
    # Normalize slashes in filename
    img_filename = row['Image'].strip().replace("\\", "/")
    label = row['Text']

    # Create full image path
    img_path = os.path.normpath(os.path.join(base_dir, img_filename))

    print(f"Looking for: {img_path}")
    print("Exists?", os.path.exists(img_path))

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"❌ Could not load image: {img_path}")
        continue

    plt.subplot(1, 5, i + 1)
    plt.imshow(img, cmap='gray')
    plt.title(f"Label:\n{label}", fontsize=8)
    plt.axis('off')

plt.tight_layout()
plt.show()


Looking for: E:\OCR_project\Datasets\archive\AHTD3A0001_Para1_3.jpg
Exists? False
❌ Could not load image: E:\OCR_project\Datasets\archive\AHTD3A0001_Para1_3.jpg
Looking for: E:\OCR_project\Datasets\archive\AHTD3A0001_Para1_4.jpg
Exists? False
❌ Could not load image: E:\OCR_project\Datasets\archive\AHTD3A0001_Para1_4.jpg
Looking for: E:\OCR_project\Datasets\archive\AHTD3A0005_Para1_1.jpg
Exists? False
❌ Could not load image: E:\OCR_project\Datasets\archive\AHTD3A0005_Para1_1.jpg
Looking for: E:\OCR_project\Datasets\archive\AHTD3A0005_Para1_2.jpg
Exists? False
❌ Could not load image: E:\OCR_project\Datasets\archive\AHTD3A0005_Para1_2.jpg
Looking for: E:\OCR_project\Datasets\archive\AHTD3A0005_Para1_3.jpg
Exists? False
❌ Could not load image: E:\OCR_project\Datasets\archive\AHTD3A0005_Para1_3.jpg


<Figure size 1500x600 with 0 Axes>

: 

In [12]:
print(df['Image'].head(5))


0                         0
1    AHTD3A0001_Para1_3.jpg
2    AHTD3A0001_Para1_4.jpg
3    AHTD3A0005_Para1_1.jpg
4    AHTD3A0005_Para1_2.jpg
Name: Image, dtype: object


In [8]:
import pandas as pd
import os

# Set your paths
train_csv = 'TrainLabels.csv'
train_dir = 'e:/OCR_project/Datasets/archive/Train_deskewed/Train_deskewed/AHTD3A1168_Para3_2.jpg'

# Load CSV
df = pd.read_csv(train_csv)

# Check which images are missing
missing = []
for img_name in df['Image']:
    full_path = os.path.join(train_dir, img_name)
    if not os.path.exists(full_path):
        missing.append(img_name)

print(f"❌ Missing {len(missing)} images:")
for img in missing[:10]:
    print(img)


❌ Missing 1902 images:
0
AHTD3A0001_Para1_3.jpg
AHTD3A0001_Para1_4.jpg
AHTD3A0005_Para1_1.jpg
AHTD3A0005_Para1_2.jpg
AHTD3A0005_Para1_3.jpg
AHTD3A0047_Para1_1.jpg
AHTD3A0047_Para1_2.jpg
AHTD3A0047_Para1_3.jpg
AHTD3A0047_Para1_4.jpg
